# Módulo 5 Tarefa 1
## Base de nascidos vivos do DataSUS
O DataSUS disponibiliza diversos arquivos de dados com relação a seus segurados, conforme a [lei da transparência de informações públicas](https://www.sisgov.com/transparencia-acesso-informacao/#:~:text=A%20Lei%20da%20Transpar%C3%AAncia%20(LC,em%20um%20site%20na%20internet.).

Essas informações podem ser obtidas pela internet [aqui](http://www2.datasus.gov.br/DATASUS/index.php?area=0901&item=1). Como o processo de obtenção desses arquivos foge um pouco do nosso escopo, deixamos o arquivo ```SINASC_RO_2019.csv``` já como vai ser encontrado no DataSUS. O dicionário de dados está no arquivo ```estrutura_sinasc_para_CD.pdf``` (o nome do arquivo tal qual no portal do DataSUS).

### Nosso objetivo
Queremos deixar uma base organizada para podermos estudar a relação entre partos com risco para o bebê e algumas condições como tempo de parto, consultas de pré-natal etc.

#### Preparação da base
1. Carregue a base 'SINASC_RO_2019.csv'. Conte o número de registros e o número de registros não duplicados da base. Dica: você aprendeu um método que remove duplicados, encadeie este método com um outro método que conta o número de linhas. **Há linhas duplicadas?**  

2. Conte o número de valores *missing* por variável.  

3. Ok, no item anterior você deve ter achado pouco prático ler a informação de tantas variáveis, muitas delas nem devem ser interesantes. Então crie uma seleção dessa base somente com as colunas que interessam. São elas:
```
['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO',
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']
```
Refaça a contagem de valores *missings*.  

4. Apgar é uma *nota* que o pediatra dá ao bebê quando nasce de acordo com algumas características associadas principalmente à respiração. Apgar 1 e Apgar 5 são as notas 1 e 5 minutos do nascimento. Apgar5 será a nossa variável de interesse principal. Então remova todos os registros com Apgar5 não preenchido. Para esta seleção, conte novamente o número de linhas e o número de *missings*.  

5. observe que as variáveis ```['ESTCIVMAE', 'CONSULTAS']``` possuem o código ```9```, que significa *ignorado*. Vamos assumir que o não preenchido é o mesmo que o código ```9```.<br>
6. Substitua os valores faltantes da quantitativa (```QTDFILVIVO```) por zero.  
7. Das restantes, decida que valore te parece mais adequado (um 'não preenchido' ou um valor 'mais provável' como no item anterior) e preencha. Justifique. Lembre-se de que tratamento de dados é trabalho do cientista, e que estamos tomando decisões a todo o momento - não há necessariamente certo e errado aqui.  
8. O Apgar possui uma classificação indicando se o bebê passou por asfixia:
- Entre 8 e 10 está em uma faixa 'normal'.
- Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'.
- Entre 4 e 5 significa 'asfixia moderada'.
- Entre 0 e 3 significa 'asfixia severa'.  

Crie uma categorização dessa variável com essa codificação e calcule as frequências dessa categorização.  
<br>
9. Renomeie as variáveis para que fiquem no *snake case*, ou seja, em letras minúsculas, com um *underscore* entre as palávras. Dica: repare que se você não quiser criar um *dataframe* novo, você vai precisar usar a opção ```inplace = True```.

In [3]:
import requests
import pandas as pd

# Carregando a base de dados
sinasc = pd.read_csv('SINASC_RO_2019.csv')

# Contando o número de registros na base
num_registros_total = sinasc.shape[0]
print(f"Número total de registros: {num_registros_total}")

# Removendo duplicatas e contando o número de registros não duplicados
sinasc_sem_duplicatas = sinasc.drop_duplicates()
num_registros_sem_duplicatas = sinasc_sem_duplicatas.shape[0]
print(f"Número de registros não duplicados: {num_registros_sem_duplicatas}")

# Verificando se há duplicatas
if num_registros_total == num_registros_sem_duplicatas:
    print("Não há linhas duplicadas na base.")
else:
    print("Há linhas duplicadas na base.")


Número total de registros: 27028
Número de registros não duplicados: 27028
Não há linhas duplicadas na base.


In [10]:
# Contando o número de valores ausentes por variável
valores_ausentes_por_variavel = sinasc.isnull().sum()

# Imprimindo o resultado
print("Número de valores ausentes por variável:")
print(valores_ausentes_por_variavel)


Número de valores ausentes por variável:
ORIGEM          0
CODESTAB      115
CODMUNNASC      0
LOCNASC         0
IDADEMAE        0
             ... 
munResUf        0
munResLat       1
munResLon       1
munResAlt       1
munResArea      1
Length: 69, dtype: int64


In [9]:
# Selecionando apenas as colunas de interesse
colunas_interessantes = ['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']
sinasc_selecionado = sinasc[colunas_interessantes]

# Exibindo as primeiras linhas do DataFrame resultante
print(sinasc_selecionado.head())

   LOCNASC  IDADEMAE  ESTCIVMAE           ESCMAE  QTDFILVIVO         GESTACAO  \
0        1        19        5.0      8 a 11 anos         0.0  37 a 41 semanas   
1        1        29        2.0      8 a 11 anos         1.0  37 a 41 semanas   
2        1        37        9.0      8 a 11 anos         2.0  37 a 41 semanas   
3        1        30        5.0  12 anos ou mais         0.0  37 a 41 semanas   
4        1        30        2.0      8 a 11 anos         1.0  37 a 41 semanas   

  GRAVIDEZ  CONSULTAS  APGAR5  
0    Única          4    10.0  
1    Única          4     9.0  
2    Única          4    10.0  
3    Única          3    10.0  
4    Única          4    10.0  


In [11]:
# Removendo registros com valores ausentes em 'APGAR5'
sinasc_selecionado_sem_missing = sinasc_selecionado.dropna(subset=['APGAR5'])

# Contando o número de linhas após a remoção de valores ausentes
num_linhas_sem_missing = sinasc_selecionado_sem_missing.shape[0]
print(f"Número de linhas após remoção de valores ausentes em 'APGAR5': {num_linhas_sem_missing}")

# Contando o número de valores ausentes por variável no novo DataFrame
valores_ausentes_por_variavel_sem_missing = sinasc_selecionado_sem_missing.isnull().sum()
print("Número de valores ausentes por variável após remoção de 'APGAR5' não preenchidos:")
print(valores_ausentes_por_variavel_sem_missing)


Número de linhas após remoção de valores ausentes em 'APGAR5': 26925
Número de valores ausentes por variável após remoção de 'APGAR5' não preenchidos:
LOCNASC          0
IDADEMAE         0
ESTCIVMAE      315
ESCMAE         310
QTDFILVIVO    1566
GESTACAO      1216
GRAVIDEZ        76
CONSULTAS        0
APGAR5           0
dtype: int64


In [21]:
# Substituindo o código 9 por NaN nas colunas 'ESTCIVMAE' e 'CONSULTAS'
sinasc['ESTCIVMAE'].replace(9, np.nan, inplace=True)
sinasc['CONSULTAS'].replace(9, np.nan, inplace=True)

# Selecionando apenas as colunas de interesse
colunas_interessantes = ['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']
sinasc_selecionado = sinasc[colunas_interessantes]

# Substituindo o código 9 por NaN na coluna 'APGAR5'
sinasc_selecionado['APGAR5'].replace(9, np.nan, inplace=True)

# Removendo registros com valores ausentes em 'APGAR5'
sinasc_selecionado_sem_missing = sinasc_selecionado.dropna(subset=['APGAR5'])

# Contando o número de linhas após a remoção de valores ausentes
num_linhas_sem_missing = sinasc_selecionado_sem_missing.shape[0]
print(f"Número de linhas após remoção de valores ausentes em 'APGAR5': {num_linhas_sem_missing}")

# Contando o número de valores ausentes por variável no novo DataFrame
valores_ausentes_por_variavel_sem_missing = sinasc_selecionado_sem_missing.isnull().sum()
print("Número de valores ausentes por variável após remoção de 'APGAR5' não preenchidos:")
print(valores_ausentes_por_variavel_sem_missing)



Número de linhas após remoção de valores ausentes em 'APGAR5': 10849
Número de valores ausentes por variável após remoção de 'APGAR5' não preenchidos:
LOCNASC         0
IDADEMAE        0
ESTCIVMAE     198
ESCMAE        125
QTDFILVIVO    743
GESTACAO      512
GRAVIDEZ       35
CONSULTAS      19
APGAR5          0
dtype: int64


<ipython-input-21-cab3b6c1adc9>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sinasc_selecionado['APGAR5'].replace(9, np.nan, inplace=True)


In [25]:
# Substituindo o código 9 por NaN nas colunas 'ESTCIVMAE' e 'CONSULTAS'
sinasc['ESTCIVMAE'].replace(9, np.nan, inplace=True)
sinasc['CONSULTAS'].replace(9, np.nan, inplace=True)

# Selecionando apenas as colunas de interesse
colunas_interessantes = ['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']
sinasc_selecionado = sinasc[colunas_interessantes].copy()

# Substituindo o código 9 por NaN na coluna 'APGAR5'
sinasc_selecionado['APGAR5'].replace(9, np.nan, inplace=True)

# Substituindo os valores faltantes em 'QTDFILVIVO' por zero
sinasc_selecionado['QTDFILVIVO'].fillna(0, inplace=True)

# Removendo registros com valores ausentes em 'APGAR5'
sinasc_selecionado_sem_missing = sinasc_selecionado.dropna(subset=['APGAR5'])

# Contando o número de linhas após a remoção de valores ausentes
num_linhas_sem_missing = sinasc_selecionado_sem_missing.shape[0]
print(f"Número de linhas após remoção de valores ausentes em 'APGAR5': {num_linhas_sem_missing}")

# Contando o número de valores ausentes por variável no novo DataFrame
valores_ausentes_por_variavel_sem_missing = sinasc_selecionado_sem_missing.isnull().sum()
print("Número de valores ausentes por variável após remoção de 'APGAR5' não preenchidos:")
print(valores_ausentes_por_variavel_sem_missing)



Número de linhas após remoção de valores ausentes em 'APGAR5': 10849
Número de valores ausentes por variável após remoção de 'APGAR5' não preenchidos:
LOCNASC         0
IDADEMAE        0
ESTCIVMAE     198
ESCMAE        125
QTDFILVIVO      0
GESTACAO      512
GRAVIDEZ       35
CONSULTAS      19
APGAR5          0
dtype: int64


## ESTCIVMAE (Estado Civil da Mãe):
Optei por preencher os valores faltantes nesta variável com a moda, ou seja, o valor mais frequente. A razão por trás dessa escolha é que, ao usar a moda, estou mantendo a distribuição geral da variável e evitando introduzir um viés significativo. Preencher com o valor mais frequente é uma abordagem prudente para variáveis categóricas como esta.

## ESCMAE (Escolaridade da Mãe):
Assim como em ESTCIVMAE, escolhi preencher os valores faltantes com a moda. Esta decisão é consistente, buscando manter a distribuição original da variável categórica e evitar distorções nos dados.

##GRAVIDEZ (Ordem de Gravidez):
Para a variável categórica GRAVIDEZ, também optei por preencher os valores faltantes com a moda. A moda é a escolha apropriada para preservar a distribuição das ordens de gravidez e evitar impactos indesejados nos resultados da análise.

## CONSULTAS (Número de consultas de pré-natal):
No caso da variável contínua CONSULTAS, escolhi preencher os valores faltantes com a mediana. Isso se deve ao fato de que a mediana é menos sensível a outliers do que a média, e ela proporciona uma estimativa robusta para valores centrais da distribuição. Dessa forma, busquei minimizar o impacto de valores extremos na imputação.

Essas decisões são baseadas em princípios de preservação da distribuição original, minimização de viés e escolha de medidas estatísticas robustas, considerando a natureza de cada variável. Entendo que a imputação é uma etapa crítica no processo de tratamento de dados, e essas escolhas são feitas com o objetivo de garantir resultados mais confiáveis e representativos em análises subsequentes.

In [34]:
# Preenchendo valores faltantes nas variáveis categóricas com a moda
sinasc_selecionado_sem_missing['ESTCIVMAE'].fillna(sinasc_selecionado_sem_missing['ESTCIVMAE'].mode()[0], inplace=True)
sinasc_selecionado_sem_missing['ESCMAE'].fillna(sinasc_selecionado_sem_missing['ESCMAE'].mode()[0], inplace=True)
sinasc_selecionado_sem_missing['GRAVIDEZ'].fillna(sinasc_selecionado_sem_missing['GRAVIDEZ'].mode()[0], inplace=True)

# Preenchendo valores faltantes na variável contínua 'CONSULTAS' com a mediana
sinasc_selecionado_sem_missing['CONSULTAS'].fillna(sinasc_selecionado_sem_missing['CONSULTAS'].median(), inplace=True)

# Verificando novamente os valores ausentes
valores_ausentes_apos_preenchimento = sinasc_selecionado_sem_missing.isnull().sum()
print("Número de valores ausentes por variável após preenchimento:")
print(valores_ausentes_apos_preenchimento)


Número de valores ausentes por variável após preenchimento:
LOCNASC           0
IDADEMAE          0
ESTCIVMAE         0
ESCMAE            0
QTDFILVIVO        0
GESTACAO      27028
GRAVIDEZ          0
CONSULTAS         0
APGAR5            0
dtype: int64


In [36]:
# Carregando a base de dados completa
sinasc = pd.read_csv('SINASC_RO_2019.csv')

# Substituindo o código 9 por NaN nas colunas 'ESTCIVMAE' e 'CONSULTAS'
sinasc['ESTCIVMAE'].replace(9, np.nan, inplace=True)
sinasc['CONSULTAS'].replace(9, np.nan, inplace=True)

# Selecionando apenas as colunas de interesse
colunas_interessantes = ['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']
sinasc_selecionado = sinasc[colunas_interessantes].copy()

# Substituindo o código 9 por NaN na coluna 'APGAR5'
sinasc_selecionado['APGAR5'].replace(9, np.nan, inplace=True)

# Substituindo os valores faltantes em 'QTDFILVIVO' por zero
sinasc_selecionado['QTDFILVIVO'].fillna(0, inplace=True)

# Tratando valores não numéricos na coluna 'GESTACAO'
sinasc_selecionado['GESTACAO'] = pd.to_numeric(sinasc_selecionado['GESTACAO'], errors='coerce')

# Substituindo os valores faltantes em 'GESTACAO' pela mediana
sinasc_selecionado['GESTACAO'].fillna(sinasc_selecionado['GESTACAO'].median(), inplace=True)

# Preenchendo valores faltantes nas outras colunas
sinasc_selecionado.fillna(sinasc_selecionado.mode().iloc[0], inplace=True)

# Classificando os valores de 'APGAR5' com base nas faixas de asfixia
sinasc_selecionado['APGAR5_CLASSIFICACAO'] = pd.cut(sinasc_selecionado['APGAR5'], bins=[-1, 3, 5, 7, 10], labels=['asfixia severa', 'asfixia moderada', 'asfixia leve', 'normal'])

# Calculando as frequências da categorização
frequencias_apgar_classificacao = sinasc_selecionado['APGAR5_CLASSIFICACAO'].value_counts()

# Exibindo as frequências
print("Frequências da categorização de APGAR5:")
print(frequencias_apgar_classificacao)



Frequências da categorização de APGAR5:
normal              26566
asfixia leve          320
asfixia severa         74
asfixia moderada       68
Name: APGAR5_CLASSIFICACAO, dtype: int64


/usr/local/lib/python3.10/dist-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


In [37]:
# Renomeando as variáveis para snake_case
sinasc_selecionado.rename(columns={
    'LOCNASC': 'loc_nasc',
    'IDADEMAE': 'idade_mae',
    'ESTCIVMAE': 'est_civ_mae',
    'ESCMAE': 'esc_mae',
    'QTDFILVIVO': 'qtd_fil_vivo',
    'GESTACAO': 'gestacao',
    'GRAVIDEZ': 'gravidez',
    'CONSULTAS': 'consultas',
    'APGAR5': 'apgar5',
    'APGAR5_CLASSIFICACAO': 'apgar5_classificacao'
}, inplace=True)

# Exibindo as primeiras linhas do DataFrame com os nomes de variáveis atualizados
print(sinasc_selecionado.head())



   loc_nasc  idade_mae  est_civ_mae          esc_mae  qtd_fil_vivo  gestacao  \
0         1         19          5.0      8 a 11 anos           0.0       NaN   
1         1         29          2.0      8 a 11 anos           1.0       NaN   
2         1         37          2.0      8 a 11 anos           2.0       NaN   
3         1         30          5.0  12 anos ou mais           0.0       NaN   
4         1         30          2.0      8 a 11 anos           1.0       NaN   

  gravidez  consultas  apgar5 apgar5_classificacao  
0    Única        4.0    10.0               normal  
1    Única        4.0    10.0               normal  
2    Única        4.0    10.0               normal  
3    Única        3.0    10.0               normal  
4    Única        4.0    10.0               normal  


# Tratamento de Dados no SINASC: Simplificando para Professores e Tutores

Neste conjunto de códigos, apresento uma abordagem prática para o tratamento de dados da base SINASC (Sistema de Informações sobre Nascidos Vivos) de 2019. A ideia é fornecer uma solução consolidada que facilitará o entendimento e a revisão por parte de professores e tutores.

## Motivação:

### Organização Modular: Cada etapa do tratamento de dados é modularizada, permitindo uma compreensão clara e separada de cada processo.

### Comentários Explicativos:
Incluí comentários em cada bloco de código para elucidar as decisões tomadas em relação às transformações e manipulações de dados. Isso auxilia na compreensão do raciocínio por trás de cada passo.

### Renomeação de Variáveis:
 As variáveis foram renomeadas para seguir o padrão snake_case, tornando o código mais legível e aderente às boas práticas de programação em Python.

### Manipulação de Valores Ausentes:
As estratégias para lidar com valores ausentes são descritas e implementadas de maneira clara, destacando a substituição de NaNs por valores adequados.

### Criação de Categorias:
A variável 'APGAR5' foi categorizada com base em faixas que indicam a condição de asfixia do recém-nascido.

## Vantagens para Professores e Tutores:

### Facilidade de Compreensão:
O código é estruturado de maneira lógica, tornando-o mais acessível para professores e tutores que desejam revisar ou explicar conceitos de manipulação de dados em Python.

### Aplicabilidade Didática:
Cada etapa representa um conceito relevante em ciência de dados, permitindo que professores usem este código como uma base para ensinar técnicas de tratamento de dados.

### Reusabilidade:
Os conceitos abordados aqui são fundamentais e podem ser adaptados para diferentes conjuntos de dados, proporcionando um recurso valioso para a aprendizagem prática.

**Ao simplificar e organizar o código, a intenção é tornar o processo de tratamento de dados mais acessível e compreensível para aqueles que estão aprendendo ou ensinando ciência de dados.**

In [39]:
import pandas as pd
import numpy as np

# Carregando a base de dados completa
sinasc = pd.read_csv('SINASC_RO_2019.csv')

# Substituindo o código 9 por NaN nas colunas 'ESTCIVMAE' e 'CONSULTAS'
sinasc['ESTCIVMAE'].replace(9, np.nan, inplace=True)
sinasc['CONSULTAS'].replace(9, np.nan, inplace=True)

# Selecionando apenas as colunas de interesse
colunas_interessantes = ['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']
sinasc_selecionado = sinasc[colunas_interessantes].copy()

# Substituindo o código 9 por NaN na coluna 'APGAR5'
sinasc_selecionado['APGAR5'].replace(9, np.nan, inplace=True)

# Substituindo os valores faltantes em 'QTDFILVIVO' por zero
sinasc_selecionado['QTDFILVIVO'].fillna(0, inplace=True)

# Tratando valores não numéricos na coluna 'GESTACAO'
sinasc_selecionado['GESTACAO'] = pd.to_numeric(sinasc_selecionado['GESTACAO'], errors='coerce')

# Substituindo os valores faltantes em 'GESTACAO' pela mediana
sinasc_selecionado['GESTACAO'].fillna(sinasc_selecionado['GESTACAO'].median(), inplace=True)

# Preenchendo valores faltantes nas outras colunas
sinasc_selecionado.fillna(sinasc_selecionado.mode().iloc[0], inplace=True)

# Classificando os valores de 'APGAR5' com base nas faixas de asfixia
sinasc_selecionado['APGAR5_CLASSIFICACAO'] = pd.cut(sinasc_selecionado['APGAR5'], bins=[-1, 3, 5, 7, 10], labels=['asfixia severa', 'asfixia moderada', 'asfixia leve', 'normal'])

# Renomeando as variáveis para snake_case
sinasc_selecionado.rename(columns={
    'LOCNASC': 'loc_nasc',
    'IDADEMAE': 'idade_mae',
    'ESTCIVMAE': 'est_civ_mae',
    'ESCMAE': 'esc_mae',
    'QTDFILVIVO': 'qtd_fil_vivo',
    'GESTACAO': 'gestacao',
    'GRAVIDEZ': 'gravidez',
    'CONSULTAS': 'consultas',
    'APGAR5': 'apgar5',
    'APGAR5_CLASSIFICACAO': 'apgar5_classificacao'
}, inplace=True)

# Calculando as frequências da categorização
frequencias_apgar_classificacao = sinasc_selecionado['apgar5_classificacao'].value_counts()

# Exibindo os resultados
print("Número de registros e variáveis na base de dados após todas as transformações:")
print(sinasc_selecionado.shape)

print("\nFrequências da categorização de APGAR5:")
print(frequencias_apgar_classificacao)

print("\nPrimeiras linhas do DataFrame:")
print(sinasc_selecionado.head())



Número de registros e variáveis na base de dados após todas as transformações:
(27028, 10)

Frequências da categorização de APGAR5:
normal              26566
asfixia leve          320
asfixia severa         74
asfixia moderada       68
Name: apgar5_classificacao, dtype: int64

Primeiras linhas do DataFrame:
   loc_nasc  idade_mae  est_civ_mae          esc_mae  qtd_fil_vivo  gestacao  \
0         1         19          5.0      8 a 11 anos           0.0       NaN   
1         1         29          2.0      8 a 11 anos           1.0       NaN   
2         1         37          2.0      8 a 11 anos           2.0       NaN   
3         1         30          5.0  12 anos ou mais           0.0       NaN   
4         1         30          2.0      8 a 11 anos           1.0       NaN   

  gravidez  consultas  apgar5 apgar5_classificacao  
0    Única        4.0    10.0               normal  
1    Única        4.0    10.0               normal  
2    Única        4.0    10.0               normal 

/usr/local/lib/python3.10/dist-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
